Data preprocessing Notebook 
After converting data into right format this notebook will convert that data into parquet format to process for next notebook 
To run it on your dataset change location in block 3 according to your dataset location 

  

In [1]:
def get_holidays():
    
    try:
        import datetime
        from dateutil.relativedelta import relativedelta, TH

        import requests
        import pandas as pd
        from bs4 import BeautifulSoup

        def thursday_holidays(year, offset=0):
            link = f'https://zerodha.com/z-connect/traders-zone/holidays/trading-holidays-{year}-nse-bse-mcx'
            soup = requests.get(link)
            table = BeautifulSoup(soup.text, 'html.parser').find_all('table')
            holidays = pd.read_html(str(table))[offset]
            holidays.columns = holidays.iloc[0]
            holidays.drop(0, inplace=True)
            holidays.reset_index(drop=True, inplace=True)

            if (holidays.columns != ['Holidays', 'Date', 'Day']).all() or (holidays.columns != ['Holiday', 'Date', 'Day']).all():
                holidays = holidays.shift()
                holidays.loc[0] = holidays.columns
                holidays.columns = ['Holidays', 'Date', 'Day']
                holidays['Date'] = holidays['Date'].apply(lambda x: datetime.datetime.strptime(x,'%B %d, %Y'))
                holidays['Date'] = holidays['Date'].apply(lambda x: x.date())
                #holidays = holidays[holidays['Day'] == 'Thursday']

            elif (holidays.columns == ['Holidays', 'Date', 'Day']).all() or (holidays.columns == ['Holiday', 'Date', 'Day']).all():
                holidays['Date'] = holidays['Date'].apply(lambda x: datetime.datetime.strptime(x,'%B %d, %Y').date())
                #holidays = holidays[holidays['Day'] == 'Thursday']

            return holidays

        all_thursdays_holidays = pd.DataFrame()

        for year in range(2018, (datetime.date.today().year+1)):
            all_thursdays_holidays = all_thursdays_holidays.append(thursday_holidays(year,0))

        all_thursdays_holidays.reset_index(drop=True,inplace=True)

        return list(all_thursdays_holidays['Date'])
    
    except:
        all_thursdays_holidays = [datetime.date(2018, 1, 26), datetime.date(2018, 2, 13), datetime.date(2018, 3, 2), datetime.date(2018, 3, 29), datetime.date(2018, 3, 30), datetime.date(2018, 5, 1), datetime.date(2018, 8, 15), datetime.date(2018, 8, 22), datetime.date(2018, 9, 13), datetime.date(2018, 9, 20), datetime.date(2018, 10, 2), datetime.date(2018, 10, 18), datetime.date(2018, 11, 7), datetime.date(2018, 11, 8), datetime.date(2018, 11, 23), datetime.date(2018, 12, 25), datetime.date(2019, 3, 4), datetime.date(2019, 3, 21), datetime.date(2019, 4, 17), datetime.date(2019, 4, 19), datetime.date(2019, 4, 29), datetime.date(2019, 5, 1), datetime.date(2019, 6, 5), datetime.date(2019, 8, 12), datetime.date(2019, 8, 15), datetime.date(2019, 9, 2), datetime.date(2019, 9, 10), datetime.date(2019, 10, 2), datetime.date(2019, 10, 8), datetime.date(2019, 10, 21), datetime.date(2019, 10, 28), datetime.date(2019, 11, 12), datetime.date(2019, 12, 25), datetime.date(2020, 2, 21), datetime.date(2020, 3, 10), datetime.date(2020, 4, 2), datetime.date(2020, 4, 6), datetime.date(2020, 4, 10), datetime.date(2020, 4, 14), datetime.date(2020, 5, 1), datetime.date(2020, 5, 25), datetime.date(2020, 10, 2), datetime.date(2020, 11, 16), datetime.date(2020, 11, 30), datetime.date(2021, 1, 26), datetime.date(2021, 3, 11), datetime.date(2021, 3, 29), datetime.date(2021, 4, 2), datetime.date(2021, 4, 14), datetime.date(2021, 4, 21), datetime.date(2021, 5, 13), datetime.date(2021, 7, 21), datetime.date(2021, 8, 19), datetime.date(2021, 9, 10), datetime.date(2021, 10, 15), datetime.date(2021, 11, 4), datetime.date(2021, 11, 5)]
        return all_thursdays_holidays

In [2]:
import dateutil
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import numpy as np
import regex as re
import datetime
import calendar
from dateutil.relativedelta import relativedelta, TH
from dateutil.parser import parse

import warnings
warnings.simplefilter("ignore")

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

holidays = get_holidays()

In [3]:

# change the location according to your dataset location beaware of putting \* in the end like you can see here

all_files = pd.DataFrame(glob(r'C:\Users\deepe\OneDrive\Desktop\NLP\2021-20210705T150540Z-001\2021\May\*'), columns=['location'])
all_files['data_date'] = all_files['location'].apply(lambda x: x.split('_')[-1].split('.')[0])
all_files['data_date'] = all_files['data_date'].apply(lambda x: datetime.datetime.strptime(x.strip(),'%d%m%Y').date())
all_files.sort_values('data_date',inplace=True)
all_files.reset_index(drop=True,inplace=True)

In [5]:

all_files = all_files[:2]
all_files

,location,data_date
0,C:\Users\deepe\OneDrive\Desktop\NLP\2021-20210705T150540Z-001\2021\May\GFDLNFO_BACKADJUSTED_01012021.csv,2021-01-01
1,C:\Users\deepe\OneDrive\Desktop\NLP\2021-20210705T150540Z-001\2021\May\GFDLNFO_BACKADJUSTED_04012021.csv,2021-01-04


In [6]:
def get_nearest_expiry(date, month_offset):

    year = date.year
    month = date.month + month_offset
    day = date.day

    if month > 12:
        month = month-12
        year = year+1

    last_day_of_the_month = datetime.date(year, month, calendar.monthrange(year, month)[1])
    last_thursday_of_the_month = last_day_of_the_month + relativedelta(weekday=TH(-1))

    if date <= last_thursday_of_the_month:
        nearest_expiry = last_thursday_of_the_month

    elif date > last_thursday_of_the_month:
        if (month+1) <= 12:
            last_day_of_the_next_month = datetime.date(
                year, month+1, calendar.monthrange(year, month+1)[1])
            last_thursday_of_the_next_month = last_day_of_the_next_month + \
                relativedelta(weekday=TH(-1))
            nearest_expiry = last_thursday_of_the_next_month
        else:
            last_day_of_the_next_month = datetime.date(year, 1, calendar.monthrange(year, 1)[1])
            last_thursday_of_the_next_month = last_day_of_the_next_month + \
                relativedelta(weekday=TH(-1))
            nearest_expiry = last_thursday_of_the_next_month

    while True:
        
        if nearest_expiry in holidays:
            nearest_expiry = nearest_expiry - datetime.timedelta(days=1)
            continue
        else:
            break

    return nearest_expiry


def get_instrument_params(x):
    
    day = None
    nearest_expiry = None
    data_date = x['date']
    ticker = x['ticker']
    flag = 0
    month_offsets = {'I':0,'II':1,'III':2}

    try:
        try:
            #ACC-III, BAJAJ-AUTO-I, L&TFH-I, M&M-I, S&P500-I
            temp = re.findall(r'([A-Z]{1,10}&[A-Z]{1,10}|[A-Z]{1,10}-[A-Z]{1,10}|[A-Z]{1,10}[0-9]{1,2}[A-Z]{1,10}|[A-Z]{1,10}|[A-Z]{1,10}&[A-Z]{1,10}[0-9]{1,3}|[A-Z]{1,9}[0-9]{1,3})(-)([I]{1,3})',ticker)[0]
            instrument_name, expiry_type = temp[0],temp[-1]
            instrument_type = 'FUT'
            flag = 1
            strike_price = np.NaN

        except:
            try:
                #ACC19FEB1240PE
                instrument_name, year, month, strike_price, instrument_type = re.findall(r'([A-Z]{1,10}&[A-Z]{1,10}|[A-Z]{1,10}-[A-Z]{1,10}|[A-Z]{1,10}[0-9]{1,2}[A-Z]{1,10}|[A-Z]{1,10})(17|18|19|20|21)(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)([0-9]{1,5})(CE|PE)',ticker)[0]
                expiry_type = None

            except:
                try:
                    #ACC24SEP201000CE
                    instrument_name, day, month, year, strike_price, instrument_type = re.findall(r'([A-Z]{1,10}&[A-Z]{1,10}|[A-Z]{1,10}-[A-Z]{1,10}|[A-Z]{1,10}[0-9]{1,2}[A-Z]{1,10}|[A-Z]{1,10})([0-9]{2})(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)(17|18|19|20|21)([0-9]{1,5})(CE|PE)',ticker)[0]
                    expiry_type = None

                except:
                    try:
                        #NIFTY19DECCE10000
                        instrument_name, year, month, instrument_type, strike_price = re.findall(r'([A-Z]{1,10}&[A-Z]{1,10}|[A-Z]{1,10}-[A-Z]{1,10}|[A-Z]{1,10}[0-9]{1,2}[A-Z]{1,10}|[A-Z]{1,10})(17|18|19|20|21)(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)(CE|PE)([0-9]{1,5})',ticker)[0]
                        expiry_type = None

                    except:
                        try:
                            #NIFTY27JUN2411500PE
                            instrument_name, day, month, year, strike_price, instrument_type = re.findall(r'(NIFTY|BANKNIFTY)([0-9]{2})(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)(22|23|24)([0-9]{1,5})(CE|PE)',ticker)[0] 
                            expiry_type = None

                        except:
                            try:
                                #OPTIDX_NIFTY_25JUN2020_CE_10000
                                instrument_name, day, month, year, instrument_type, strike_price = re.findall(r'(NIFTY|BANKNIFTY)_([0-9]{2})(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)(2019|2020|2021)_(CE|PE)_([0-9]{1,5})',ticker)[0]
                                expiry_type = None
                                year = year[-2:]

                            except:
                                try:
                                    #TORNTPOWER19AUGFUT
                                    instrument_name,year,month,instrument_type = re.findall(r'([A-Z]{3,12})(19|20)(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)(FUT)',ticker)[0]
                                    expiry_type = None
                                    strike_price = np.NaN

                                except:
                                    try:
                                        #----------------------------------------------------------------------------
                                        #ADANIENT25JUN20137 this one not required hence fill the values with na later
                                        ticker = re.findall(r'([A-Z]{1,10}&[A-Z]{1,10}|[A-Z]{1,10}-[A-Z]{1,10}|[A-Z]{1,10}[0-9]{1,2}[A-Z]{1,10}|[A-Z]{1,10})([0-9]{2})(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)(17|18|19|20|21)([0-9]{1,5})',ticker)[0]
                                        strike_price = np.NaN
                                        instrument_type = np.NaN
                                        nearest_expiry = np.NaN
                                        instrument_name = np.NaN
                                        expiry_type = np.NaN  

                                    except:
                                        try:
                                            #----------------------------------------------------------------------------
                                            #ADANIPOWER19JAN57 this one not required hence fill the values with na later
                                            ticker = re.findall(r'([A-Z]{1,10}&[A-Z]{1,10}|[A-Z]{1,10}-[A-Z]{1,10}|[A-Z]{1,10}[0-9]{1,2}[A-Z]{1,10}|[A-Z]{1,10})(17|18|19|20|21)(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)([0-9]{1,5})',ticker)[0]
                                            strike_price = np.NaN
                                            instrument_type = np.NaN
                                            nearest_expiry = np.NaN
                                            instrument_name = np.NaN
                                            expiry_type = np.NaN
                                        except:
                                            ticker = np.NaN
                                            strike_price = np.NaN
                                            instrument_type = np.NaN
                                            nearest_expiry = np.NaN
                                            instrument_name = np.NaN
                                            expiry_type = np.NaN

        #options expiry computation
        if (day == None) & (nearest_expiry == None) & (flag == 0):
            year = int(year)
            month = datetime.datetime.strptime(month,'%b').month
            nearest_expiry = datetime.date(year,month,calendar.monthrange(year, month)[1]) + relativedelta(weekday=TH(-1))
            nearest_expiry = datetime.datetime.strptime(str(nearest_expiry),'00%y-%m-%d').date()
            while True:
                if nearest_expiry in holidays:
                    nearest_expiry = nearest_expiry - datetime.timedelta(days = 1)
                    continue
                else:
                    break

        #banknifty/nifty weekly options expiry computation
        elif (day != None) & (nearest_expiry == None) & ((instrument_type == 'CE') or (instrument_type == 'PE')):
            nearest_expiry = datetime.datetime.strptime(day+month+year,'%d%b%y').date()

        #futures expiry computation
        elif (day == None) & (nearest_expiry == None) & (instrument_type == 'FUT'):
            nearest_expiry = get_nearest_expiry(data_date,month_offsets[expiry_type])

        return strike_price, instrument_type, nearest_expiry, instrument_name, expiry_type
    except:
        print('Error in: ',ticker)

In [7]:
eliminated_files = []

In [9]:
%%time
print(datetime.datetime.now().time())

for index, row in all_files.iterrows():
    
    print(index)
    try:
    
        data = pd.read_csv(row['location'])
        data.columns = data.columns.str.lower()
        data.columns = data.columns.str.strip()
        data.columns = data.columns.str.replace(' ', '_')

        try:
            data['time'] = data['time'].apply(lambda x: x.strip())
        except:
            data['time'] = data['time'].apply(lambda x: str(x).strip())

        try:
            data['date'] = data['date'].apply(lambda x: x.strip())
        except:
            data['date'] = data['date'].apply(lambda x: str(x).strip())


        data = data.rename(columns={'openinterest':'open_interest'})
        data = data.rename(columns={'opne_interest':'open_interest'})
        data = data.rename(columns={'dtae':'date'})
        data = data.rename(columns={'higgh':'high'})

        data = data[['ticker', 'date', 'time', 'open', 'high', 'low', 'close', 'volume', 'open_interest']]

        try:
            data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y').date())
        except:
            try:
                data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x,'%d-%m-%Y').date())
            except:        
                try:
                    data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y').date() if x!='nan' else np.NaN)
                except:
                    data['time'] = np.where(data['time'] == '#VALUE!', data['date'].apply(lambda x: x.split(' ')[-1]),data['time'])
                    data['date'] = data['date'].apply(lambda x: x.split(' ')[0])
                    data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y').date())

        try:
            data['time'] = data['time'].apply(lambda x: datetime.datetime.strptime(x,'%H:%M:%S').time().replace(second=0, microsecond=0))
        except:
            try:
                data['time'] = data['time'].apply(lambda x: datetime.datetime.strptime(x,'%H:%M:%S').time().replace(second=0, microsecond=0) if x!='nan' else np.NaN)
            except:
                data['time'] = data['time'].apply(lambda x: datetime.datetime.strptime(x,'%H:%M:%S').time().replace(second=0, microsecond=0) if len(x)>5 else datetime.datetime.strptime(str(x)+':00','%H:%M:%S').time().replace(second=0, microsecond=0))

        try:
            data['datetime'] = data[['date','time']].apply(lambda x: datetime.datetime.combine(x['date'],x['time']),axis=1)
        except:
            try:
                data['datetime'] = data[['date','time']].apply(lambda x: datetime.datetime.combine(x['date'],x['time']) if ((x['date']!='nan') and (x['time']!='nan')) else np.NaN,axis=1)
            except:
                data = data.dropna(subset=['time'])
                data['datetime'] = data[['date','time']].apply(lambda x: datetime.datetime.combine(x['date'],x['time']),axis=1)

        data['open'] = data['open'].astype('float')
        data['high'] = data['high'].astype('float')
        data['low'] = data['low'].astype('float')
        data['close'] = data['close'].astype('float')
        data['volume'] = data['volume'].astype('int')
        data['open_interest'] = data['open_interest'].astype('int')
        data['ticker'] = data['ticker'].apply(lambda x: (x.strip()).split('.')[0])

        data['temp'] = data.apply(get_instrument_params,axis=1)
        data['strike_price'] = data['temp'].apply(lambda x: x[0])
        data['instrument_type'] = data['temp'].apply(lambda x: x[1])
        data['expiry_date'] = data['temp'].apply(lambda x: x[2])
        data['instrument_name'] = data['temp'].apply(lambda x: x[3])
        data['expiry_type'] = data['temp'].apply(lambda x: x[4])
        data = data[['ticker', 'datetime', 'date', 'time', 'open', 'high', 'low', 'close', 'volume', 'open_interest', 'strike_price', 'instrument_type', 'expiry_date', 'instrument_name', 'expiry_type']]
        data['strike_price'] = data['strike_price'].apply(float)

        data = data.dropna(subset=['instrument_name'])
#         data.to_parquet(f"/Users/entirety/Documents/Entirety - AMC/Data/variable_nfo_data_2016-21/all_instruments_nfo_data_{row['data_date']}.parquet")
        data.to_parquet(fr"C:\Users\deepe\OneDrive\Desktop\NLP\bro_{row['data_date']}.parquet")


    except Exception as e:
        print(e)
        print(row['location'])
        eliminated_files.append(row['location'])

print(datetime.datetime.now().time())

14:21:40.617001
0
time data '01-01-2021' does not match format '%d/%m/%Y'
C:\Users\deepe\OneDrive\Desktop\NLP\2021-20210705T150540Z-001\2021\May\GFDLNFO_BACKADJUSTED_01012021.csv
1
14:22:36.472118
Wall time: 55.9 s


In [10]:
print('hello')

hello
